# Plot shot gathers 

Daniel Koehn, Anna Jegen

Kiel, 08th of February 2021

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 150
import matplotlib.pyplot as plt

import numpy as np
import segyio
from numba import jit

**Read field data and header info**

In [ ]:
# Define shot number
shotno = 1

# Define path and filename of field data
rawdata='DENISE_MARMOUSI_y.su.shot' + str(shotno)
path2data='DENISE/'
rawdatafile=path2data + rawdata

# Import seismic shot gathers & headers
with segyio.su.open(rawdatafile,endian='little', ignore_geometry=True) as f:
        
        # Get basic attributes
        shot = f.trace.raw[:]  # Get all data into memory (could cause on big files)
        
        # Get number of traces ntr & number of samples ns
        ntr, ns = np.shape(shot)
        
        # Get time sample interval dt and define time axis t
        dt = 1e-6 * f.header[0][segyio.TraceField.TRACE_SAMPLE_INTERVAL]
        t = np.arange(0.,dt*(ns+1),dt)
        
        # Get offset, CDP and SourceX for each trace
        offset = np.zeros(ntr)
        SourceX_shot = np.zeros(ntr)
        
        for i in range (ntr):
            
            offset[i] = f.header[i][segyio.TraceField.offset] / 1000
            SourceX_shot[i] = f.header[i][segyio.TraceField.SourceX]

**Plot CDP gather**

In [ ]:
def plot_shot(shot, title, pclip=1.0):  
    vmax = pclip * np.max(np.abs(shot)) 
    vmin = - vmax 
    plt.figure(figsize=(10,5), dpi=100) 
    plt.imshow(shot.T, extent=[np.min(offset),np.max(offset),np.max(t),np.min(t)], cmap='Greys', vmin=vmin, vmax=vmax)  
    plt.xlabel('Offset [m]')
    plt.ylabel('Time [s]')
    plt.title(title) 
    plt.axis('auto')

In [ ]:
plot_shot(shot, 'Shot gather no. ' + str(shotno), pclip=1e0)
plt.savefig('Seis_shot_' + str(shotno) + '.pdf', bbox_inches='tight', format='pdf')
#plt.show()